In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
num_gpus_available = len(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", num_gpus_available)
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("complete_preprocessing_stp_frsc.csv",encoding = "utf-8",sep = ',')

In [ ]:
df_dev = pd.read_csv("dev_complete_preprocessing_stp_frsc.csv",encoding = "utf-8",sep = ',')

In [ ]:
df_test = pd.read_csv("test_complete_preprocessing_stp_frsc.csv",encoding = "utf-8",sep = ',')

In [ ]:
df = df.reset_index()

In [ ]:
df.drop(columns=['index'],inplace=True)

In [ ]:
df_dev = df_dev.reset_index()
df_dev.drop(columns=['index'],inplace=True)

In [ ]:
df_test = df_test.sort_values(by='review_id')
df_test = df_test.reset_index()
df_test.drop(columns=['index'],inplace=True)

In [ ]:
df

In [ ]:
df_dev

In [ ]:
df_test

In [ ]:
df['note'] = df['note'].astype('int')
df_dev['note'] = df_dev['note'].astype('int')

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
final_stopwords_list = stopwords.words('french')
tfidf = TfidfVectorizer()
features = tfidf.fit(df.commentaire.values.tolist())#.toarray()
labels = df.note
#features.shape

In [ ]:
features_values = features.transform(df.commentaire.values.tolist())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['commentaire'], df['note'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

In [ ]:
import pickle

In [ ]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df.commentaire.values.tolist())
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
svm = LinearSVC().fit(X_train_tfidf, df.note.values.tolist())

In [ ]:
pickle.dump(svm, open("svm_complete_preprocessing_data_without_stp_frsc.pickle", "wb"))

In [ ]:
X_test_counts = count_vect.transform(df_dev.commentaire.values.tolist())
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [ ]:
from sklearn.metrics import accuracy_score
score = accuracy_score(df_dev.note.values.tolist(), svm.predict(X_test_tfidf))
print(score)

In [ ]:
X_test_counts = count_vect.transform(df_test.commentaire.values.tolist())
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [ ]:
results =[]
for i in range(len(df_test)):
    results.append([df_test['review_id'].iloc[i],str((svm.predict(X_test_tfidf[i])[0]+1)/2).replace('.',',')])

In [ ]:
results

In [ ]:
def write_list_to_file(data):
    # Open the file in write mode
    with open('results_svm_tfidf_without_stp_frsc.txt', 'w') as f:
        # Iterate over the outer list
        for row in data:
            # Convert the inner list to a string with space separator
            row_str = " ".join(str(x) for x in row)
            # Write the string to the file
            f.write(f"{row_str}\n")

In [ ]:
write_list_to_file(results)